# Automated ML


In [1]:
!pip install xgboost==0.90

In [2]:
import json
import sys
import os
import numpy as np
import pandas as pd
import shutil
import joblib
import requests

from sklearn.model_selection import train_test_split

from TrainCovid19Infections import clean_data

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.run import Run
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.train.automl import AutoMLConfig

from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.train.automl import constants

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## Initialize Workspace

In [3]:
# Get current workspace from config
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

project_folder = './capstone-project'
# choose a name for experiment
experiment_name = 'Covid19VaccinationExperiment'
experiment=Experiment(ws, experiment_name)
experiment

Workspace name: wsptest
Azure region: eastus2
Subscription id: c04b3d3f-4994-454d-96ff-aa3f2050b57f
Resource group: testingMLFunctionnalities


Name,Workspace,Report Page,Docs Page
Covid19VaccinationExperiment,wsptest,Link to Azure Machine Learning studio,Link to Documentation


## Create Cluster

Get cluster if it exists else create one

In [4]:
# Create compute cluster
cpu_cluster_name = "Covid19Cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,identity_type="SystemAssigned")
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# Get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-21T22:06:18.896000+00:00', 'errors': None, 'creationTime': '2021-03-21T03:05:29.360274+00:00', 'modifiedTime': '2021-03-21T03:05:45.676021+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

I Chose a COVID-19 World Wide infections Dataset that holds a track of Covid19's propagation rate, new cases, patients with chronic deseases and death rate per country. 

Since the covid-19 pandemic is among the hottest subjects in the world, and as a member of the society being interested in such statistics calculation can help further scientists or even regular people to better understand the global effect of this deadly virus all over the world.

I used Github to download the Dataset.

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True)
    
    
    #ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,7.662200e+04,76622.000000,76622.000000,7.662200e+04,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,...,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000,76622.000000
mean,6.712233e+05,5139.614667,5077.930640,1.721450e+04,114.360497,113.334672,7965.231467,66.744952,65.736372,162.348091,...,27.694392,7.877240,5.021261,17410.995745,236.143470,7.253108,23.489801,2.546956,69.503518,0.667334
std,4.742487e+06,32213.276703,31682.106181,1.090468e+05,674.204909,652.787847,15827.741365,168.616215,141.650887,318.174727,...,12.420852,6.481445,4.369400,19622.665161,133.785524,4.296363,33.328230,2.519239,17.535769,0.246832
min,0.000000e+00,-74347.000000,-6223.000000,0.000000e+00,-1918.000000,-232.143000,0.000000,-2153.437000,-276.825000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.830000e+02,1.000000,4.429000,1.000000e+01,0.000000,0.000000,124.740000,0.061000,0.746000,1.652000,...,19.600000,3.008000,1.783000,2896.913000,140.448000,4.610000,0.000000,0.700000,66.470000,0.555000
50%,7.900500e+03,55.000000,63.429000,1.310000e+02,1.000000,0.857000,1047.361000,5.893000,7.590500,19.817000,...,29.000000,5.440000,3.212000,10727.146000,233.070000,6.930000,0.000000,2.000000,74.160000,0.737000
75%,8.796125e+04,664.000000,671.107250,1.731000e+03,11.000000,11.571000,7286.520750,56.359750,62.297000,141.677750,...,38.000000,13.260000,8.353000,25063.846000,318.949000,9.750000,47.782000,3.600000,78.490000,0.828000
max,1.236781e+08,880902.000000,739564.429000,2.722975e+06,17903.000000,14431.429000,149420.824000,8652.658000,2648.773000,2327.774000,...,48.200000,27.049000,18.493000,116935.600000,724.417000,30.530000,98.999000,13.800000,86.750000,0.957000


In [6]:
# preview the first 10 rows of the dataset
df.head(10)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
5,AFG,Asia,Afghanistan,2020-02-29,1.0,0.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
6,AFG,Asia,Afghanistan,2020-03-01,1.0,0.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
7,AFG,Asia,Afghanistan,2020-03-02,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
8,AFG,Asia,Afghanistan,2020-03-03,2.0,1.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
9,AFG,Asia,Afghanistan,2020-03-04,4.0,2.0,0.429,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511


In [7]:
# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
data = pd.concat([x,y],axis=1)
for col in data.columns:
    print (col)
data.head(10)

total_cases
new_cases_smoothed
total_deaths
new_deaths
new_deaths_smoothed
total_cases_per_million
new_cases_per_million
new_cases_smoothed_per_million
total_deaths_per_million
new_deaths_per_million
new_deaths_smoothed_per_million
reproduction_rate
icu_patients
icu_patients_per_million
hosp_patients
new_tests
total_tests
total_tests_per_thousand
positive_rate
tests_per_case
stringency_index
population
population_density
median_age
aged_65_older
aged_70_older
gdp_per_capita
extreme_poverty
cardiovasc_death_rate
diabetes_prevalence
female_smokers
male_smokers
handwashing_facilities
hospital_beds_per_thousand
life_expectancy
human_development_index
tests_units
iso_code_AFG
iso_code_AGO
iso_code_AIA
iso_code_ALB
iso_code_AND
iso_code_ARE
iso_code_ARG
iso_code_ARM
iso_code_ATG
iso_code_AUS
iso_code_AUT
iso_code_AZE
iso_code_BDI
iso_code_BEL
iso_code_BEN
iso_code_BFA
iso_code_BGD
iso_code_BGR
iso_code_BHR
iso_code_BHS
iso_code_BIH
iso_code_BLR
iso_code_BLZ
iso_code_BMU
iso_code_BOL
iso_code

,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,...,iso_code_ZMB,iso_code_ZWE,continent_0,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America,new_cases
0,1.0,0.000,0.0,0.0,0.0,0.026,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,1.0,0.000,0.0,0.0,0.0,0.026,0.000,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,1.0,0.000,0.0,0.0,0.0,0.026,0.000,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,1.0,0.000,0.0,0.0,0.0,0.026,0.000,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,1.0,0.000,0.0,0.0,0.0,0.026,0.000,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
5,1.0,0.143,0.0,0.0,0.0,0.026,0.000,0.004,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
6,1.0,0.143,0.0,0.0,0.0,0.026,0.000,0.004,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
7,1.0,0.000,0.0,0.0,0.0,0.026,0.000,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
8,2.0,0.143,0.0,0.0,0.0,0.051,0.026,0.004,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
9,4.0,0.429,0.0,0.0,0.0,0.103,0.051,0.011,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1


In [53]:
# Split data into train and test sets.
training_data,validation_data = train_test_split(data,test_size = 0.5,random_state = 42,shuffle=True)
validation_data.head(10)

,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,...,iso_code_ZMB,iso_code_ZWE,continent_0,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America,new_cases
49563,6204.00,7.43,168.00,0.00,0.14,936.52,0.00,1.12,25.36,0.00,...,0,0,0,0,0,0,1,0,0,0
69508,23.00,3.14,0.00,0.00,0.00,2.78,0.00,0.38,0.00,0.00,...,0,0,0,1,0,0,0,0,0,0
27859,221147.00,2289.57,7091.00,53.00,47.57,21217.10,156.00,219.66,680.32,5.08,...,0,0,0,0,0,1,0,0,0,1
56267,148258.00,224.14,248.00,0.00,0.29,51459.53,89.55,77.80,86.08,0.00,...,0,0,0,0,1,0,0,0,0,1
76147,17916.00,44.00,364.00,0.00,1.00,974.54,0.98,2.39,19.80,0.00,...,1,0,0,1,0,0,0,0,0,1
26375,1182.00,5.29,17.00,0.00,0.14,296.30,0.75,1.32,4.26,0.00,...,0,0,0,0,1,0,0,0,0,1
66297,8243.00,65.71,152.00,2.00,0.86,14051.35,117.62,112.02,259.11,3.41,...,0,0,0,0,0,0,0,0,1,1
48835,1083686.00,4448.86,15529.00,63.00,55.86,63244.47,257.60,259.64,906.28,3.68,...,0,0,0,0,0,1,0,0,0,1
14561,44386.00,2954.14,1112.00,100.00,88.71,30.84,1.41,2.05,0.77,0.07,...,0,0,0,0,1,0,0,0,0,1
41557,17587.00,16.29,259.00,0.00,0.57,635.12,0.00,0.59,9.35,0.00,...,0,0,0,1,0,0,0,0,0,0


In [38]:
# Create necessary folders
if "automl_training" not in os.listdir():
    os.mkdir("./automl_training")
if "data" not in os.listdir():
    os.mkdir("./data")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
if "training" not in os.listdir():
    os.mkdir("./training")
# store training_dataset into it using datastore
script_folder = './automl_training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('TrainCovid19Infections.py', script_folder)
project_folder = './pipeline-project'

## AutoML Configuration
TODO: Explain why you chose the automl settings and cofiguration you used below.
The settings used below refers to a classification task within a number of settings chosen based on the existing workspace and cluster configuration restrictions 

In [39]:
#convert the training dataset to a CSV file and store it under the training folder
training_data.to_csv('training/training_data.csv')
#Create an experiment for the AutoML testing script
exp = Experiment(workspace=ws, name="Covid19AutoMlExperiment")

# Get the dataset from the data folder
datastore.upload_files(files = ['training/training_data.csv'],
                       target_path ='./data/',
                       overwrite = True,
                       show_progress = True)
training_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,('./data/training_data.csv'))])
#training_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 1.0,
    "max_concurrent_iterations": 3,
}
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = cpu_cluster,
                             training_data = training_dataset,
                             label_column_name = 'new_cases',
                             featurization= 'auto',
                             path=project_folder,
                              model_explainability=True,
                             debug_log = "Covid_automl_errors.log",
                             **automl_settings)

Uploading an estimated of 1 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [36]:
# Experiment Submission
tag = {"Covid19Infections": "Capstone project: Covid19 AutoML Experiment"}
remote_run = experiment.submit(automl_config,tags=tag, show_output=True)

Running on remote.
No run_configuration provided, running on Covid19Cluster with default configuration
Running on remote compute: Covid19Cluster
The validation results are as follows:
Y is empty


ValidationException: ValidationException:
	Message: Failed to execute the requested operation: data/settings validation. Error details: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Y is empty",
    "message_format": "{dataName} is empty",
    "message_parameters": {
        "dataName": "Y"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "details_uri": null,
            "target": null,
            "details": [],
            "inner_error": null
        }
    ],
    "inner_error": {
        "additional_properties": {},
        

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?
Supervised Learning in general is based on labled 

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_2ada2add-c641-416f-962c-002548ae3999',
 'target': 'Covid19Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-23T11:50:21.042881Z',
 'endTimeUtc': '2021-03-23T13:22:44.875995Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'Covid19Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Covid19VaccinationExperiment","subscription_id":"c04b3d3f-4994-454d-96ff-aa3f2050b57f","resource_group":"testingMLFunctionnalities","workspace_name":"wsptest","region":"eastus2","compute_target":"Covid19Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_m

In [13]:
# List best models of HyperDrive Run and AutoML Run to compare the accuracy of the models and choose the best among them to deploy
for model in Model.list(ws):
    print(model.name)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t',tag_name,':',tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t',prop_name,':',prop)
    print("\n")

Covid19-automl-model
	 Area : Pandemic
	 Type : Classification
	 Method of execution : Auto ML




## Best Model

explaining_model_run_id = remote_run.id
print(explaining_model_run_id)
explaining_model_run = Run(experiment=experiment, run_id=explaining_model_run_id)
explaining_model_run.wait_for_completion()

In [14]:
# Retrieve and save best model.
best_automl_run, best_automl_model = remote_run.get_output()

In [15]:
# Get the metrics of the best selected run
best_run_metrics = best_automl_run.get_metrics()

# Print all metrics of the best run model
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print(best_automl_model._final_estimator)

average_precision_score_weighted 0.9999641810702895
weighted_accuracy 0.9993212052715844
AUC_macro 0.9999691244540347
average_precision_score_macro 0.999939867521063
log_loss 0.026770409255269773
matthews_correlation 0.9976873494326861
recall_score_macro 0.9987763472908391
accuracy 0.9991125339199882
precision_score_micro 0.9991125339199882
norm_macro_recall 0.9975526945816778
AUC_weighted 0.9999691244540347
precision_score_macro 0.9989110981966893
average_precision_score_micro 0.9999769862542124
recall_score_weighted 0.9991125339199882
balanced_accuracy 0.9987763472908391
f1_score_macro 0.998843467273618
recall_score_micro 0.9991125339199882
f1_score_micro 0.9991125339199882
precision_score_weighted 0.9991127371256651
f1_score_weighted 0.9991124761088764
AUC_micro 0.9999766096223297
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_2ada2add-c641-416f-962c-002548ae3999_23/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_2ada2add-c641-416f-962c-002548ae

In [16]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Covid19VaccinationExperiment,AutoML_2ada2add-c641-416f-962c-002548ae3999_23,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
joblib.dump(best_automl_model, filename="./outputs/automl-model.pkl")

['./outputs/automl-model.pkl']

In [18]:
arr = os.listdir('./outputs/')
print(arr)

['automl-model.pkl', 'model.joblib']


In [19]:
from azureml.core.model import Model
model = Model.list(ws)
print (model)

[Model(workspace=Workspace.create(name='wsptest', subscription_id='c04b3d3f-4994-454d-96ff-aa3f2050b57f', resource_group='testingMLFunctionnalities'), name=Covid19-automl-model, id=Covid19-automl-model:2, version=2, tags={'Area': 'Pandemic', 'Type': 'Classification', 'Method of execution': 'Auto ML'}, properties={})]


In [20]:
# register the best model
best_model_registered = remote_run.register_model(
                                           
                                            model_name = "Covid19-automl-model",
                                            tags={'Area': "Pandemic", 'Type': "Classification",'Method of execution':'Auto ML'},
                                            )
#print(best_model.name, best_model.id, sep='\t')
print(best_model_registered)

Model(workspace=Workspace.create(name='wsptest', subscription_id='c04b3d3f-4994-454d-96ff-aa3f2050b57f', resource_group='testingMLFunctionnalities'), name=Covid19-automl-model, id=Covid19-automl-model:3, version=3, tags={'Area': 'Pandemic', 'Type': 'Classification', 'Method of execution': 'Auto ML'}, properties={})


## Model Deployment

As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [54]:
with open('score.py') as f:
    print(f.read())

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame({"Column1": pd.Series([0], dtype="int64"), "total_cases": pd.Series([0.0], dtype="float64"), "new_cases_smoothed": pd.Series([0.0], dtype="float64"), "total_deaths": pd.Series([0.0], dtype="float64"), "new_deaths": pd.Series([0.0], dtype="float64"), "new_deaths_smoothed": pd.Series([0.0], d

In [22]:
from azureml.automl.core.shared import constants

model = Model(ws, 'Covid19-automl-model')

environment = best_automl_run.get_environment()
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_automl_run.download_file(constants.CONDA_ENV_FILE_PATH, 'environment.yml')


inference_config = InferenceConfig(entry_script='score.py', environment=environment)

deployment_config = AciWebservice.deploy_configuration(
                                                       cpu_cores=1,
                                                       memory_gb=1,
                                                       description='Covid19 new cases prediction',
                                                       auth_enabled=True,
                                                       enable_app_insights= True,
                                                       collect_model_data = True)

service = Model.deploy(workspace=ws,
                       name='aci-covid19-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-23 13:25:44+00:00 Creating Container Registry if not exists.
2021-03-23 13:25:45+00:00 Registering the environment.
2021-03-23 13:25:45+00:00 Use the existing image.
2021-03-23 13:25:46+00:00 Generating deployment configuration..
2021-03-23 13:25:50+00:00 Submitting deployment to compute..
2021-03-23 13:25:55+00:00 Checking the status of deployment aci-covid19-service..
2021-03-23 13:29:48+00:00 Checking the status of inference endpoint aci-covid19-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [48]:
from azureml.pipeline.core import PipelineEndpoint
list(ws)

TypeError: 'Workspace' object is not iterable

In [43]:
print("Key " + service.get_keys()[0])
print("Swagger URI : "+service.swagger_uri)
print("Scoring URI : "+service.scoring_uri)

ERROR:azureml.core.webservice.webservice:Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Tue, 23 Mar 2021 14:02:39 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': '6d30f7f3-7631-4f49-9126-57a2420335e5', 'x-ms-client-session-id': 'd0f7d033-2e09-4cf5-b055-9289d380199f', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.031'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"NoSuchService","message":"There is no service with name: aci-covid19-service in Subscription: c04b3d3f-4994-454d-96ff-aa3f2050b57f, ResourceGroup: testingMLFunctionnalities, Workspace: wsptest, ACR: /subscriptions/c04b3d3f-4994-454d-96ff-aa3f2050b57f/resourceGroups/testingMLFunctionna

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Tue, 23 Mar 2021 14:02:39 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': '6d30f7f3-7631-4f49-9126-57a2420335e5', 'x-ms-client-session-id': 'd0f7d033-2e09-4cf5-b055-9289d380199f', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.031'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"NoSuchService","message":"There is no service with name: aci-covid19-service in Subscription: c04b3d3f-4994-454d-96ff-aa3f2050b57f, ResourceGroup: testingMLFunctionnalities, Workspace: wsptest, ACR: /subscriptions/c04b3d3f-4994-454d-96ff-aa3f2050b57f/resourceGroups/testingM

In [40]:
# testing the endpoint
x_test = validation_data.sample(2)
y_test = y

In [41]:
Covid19DataTesting= json.dumps({'data': x_test.to_dict(orient='records')})
print(Covid19DataTesting)

{"data": [{"total_cases": 3436.0, "new_cases_smoothed": 17.714, "total_deaths": 52.0, "new_deaths": 0.0, "new_deaths_smoothed": 0.429, "total_cases_per_million": 1745.937, "new_cases_per_million": 19.309, "new_cases_smoothed_per_million": 9.001, "total_deaths_per_million": 26.423, "new_deaths_per_million": 0.0, "new_deaths_smoothed_per_million": 0.218, "reproduction_rate": 0.92, "icu_patients": 0, "icu_patients_per_million": 0, "hosp_patients": 0, "new_tests": 0, "total_tests": 0, "total_tests_per_thousand": 0, "positive_rate": 0, "tests_per_case": 0, "stringency_index": 0.0, "population": 1967998.0, "population_density": 66.191, "median_age": 19.4, "aged_65_older": 3.002, "aged_70_older": 1.565, "gdp_per_capita": 1548.675, "extreme_poverty": "67.1", "cardiovasc_death_rate": 382.474, "diabetes_prevalence": 2.42, "female_smokers": 0, "male_smokers": 0, "handwashing_facilities": 6.403, "hospital_beds_per_thousand": 0.0, "life_expectancy": 58.32, "human_development_index": 0.48, "tests_un

In [52]:
headers = {'Content-type': 'application/json'}
#headers['Authorization'] = f'Bearer {service.get_keys()[0]}'
headers['Authorization'] = f'Bearer ScgS3H8AP6nX2ARiBqLbZ9T0Tezmm8jv'

# Make the request and display the response
response = requests.post('http://d8bef598-b001-416d-b965-40b48948a8cb.eastus2.azurecontainer.io/score', Covid19DataTesting, headers=headers)
print('Prediction :', response.text)

# Print original labels
print('True Values :', y_test.values)

Prediction : "{\"error\": \"DataException:\\n\\tMessage: Expected column(s) new_cases not found in fitted data.\\n\\tInnerException: None\\n\\tErrorResponse \\n{\\n    \\\"error\\\": {\\n        \\\"code\\\": \\\"UserError\\\",\\n        \\\"message\\\": \\\"Expected column(s) new_cases not found in fitted data.\\\",\\n        \\\"target\\\": \\\"X\\\",\\n        \\\"inner_error\\\": {\\n            \\\"code\\\": \\\"BadArgument\\\",\\n            \\\"inner_error\\\": {\\n                \\\"code\\\": \\\"MissingColumnsInData\\\"\\n            }\\n        },\\n        \\\"reference_code\\\": \\\"17049f70-3bbe-4060-a63f-f06590e784e5\\\"\\n    }\\n}\"}"
True Values : [0 0 0 ... 1 1 1]


In [27]:
print(service.get_logs())

2021-03-23T13:29:42,151702300+00:00 - rsyslog/run 
2021-03-23T13:29:42,154233700+00:00 - gunicorn/run 
2021-03-23T13:29:42,180991200+00:00 - iot-server/run 
2021-03-23T13:29:42,222595300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [28]:
#service.delete()
#cpu_cluster.delete()